# This notebook shows how to manage the events table with Spark

We first need to the kudu jars to the spark session

In [9]:
%%configure -f
{ "conf": {"spark.jars.packages": "org.apache.kudu:kudu-spark2_2.11:1.4.0" }}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
75,None,spark,idle,,,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
73,None,spark,idle,,,
75,None,spark,idle,,,✔


In [10]:
sc

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@bff6ed0

In [26]:
import org.apache.kudu.spark.kudu._
import org.apache.spark.sql.functions._

//we need to define a udf function for converting two strings into a map
val asMap = udf((keys:String, values: String) => keys.split("#").zip(values.split("#")).toMap)

//then we define a first dataframe df1 associated to the kudu table
val df1 = spark.sqlContext.read.options(Map("kudu.master" -> "master:7051","kudu.table" -> "Events")).kudu

//the next dataframe contains an additional column of type Map containing a map for the two arrays: keys and values
val df2 = df1.withColumn("tags", asMap($"attributesKeys",$"attributesValues"))

df2: org.apache.spark.sql.DataFrame = [id: string, ts: bigint ... 12 more fields]

In [27]:
df2.head

res50: org.apache.spark.sql.Row = [class it.gov.daf.kafkaProducers.eventConverters.TorinoParkingConverter.1505819054000.2,1505819054000,1,null,1.0,0,null,null,http://opendata.5t.torino.it/get_pk,45.04289-7.67754,[B@7fa329bb,name#tendence#tags#status#metric#value,D'AZEGLIO GALILEI#1#name,status,tendence#1#free#105,Map(name -> D'AZEGLIO GALILEI, tendence -> 1, tags -> name,status,tendence, status -> 1, metric -> free, value -> 105)]

In [28]:
//let's show a couple of rows of df2
//df2.select("id", "ts", "tags").filter($"id"=="twitter-9181205099291776")
//df2.filter("$id == twitter-9181205099291776")
df2.filter("ts == 1505819054000").count()

res54: Long = 13

In [29]:
df2.count()

res55: Long = 32701458

In [7]:
df2.wh

res13: org.apache.spark.sql.Row = [class it.gov.daf.kafkaProducers.eventConverters.TorinoParkingConverter.1505819054000.2,1505819054000,1,null,1.0,0,null,null,http://opendata.5t.torino.it/get_pk,45.04289-7.67754,[B@1b15812a,name#tendence#tags#status#metric#value,D'AZEGLIO GALILEI#1#name,status,tendence#1#free#105,Map(name -> D'AZEGLIO GALILEI, tendence -> 1, tags -> name,status,tendence, status -> 1, metric -> free, value -> 105)]

In [6]:
%%cleanup -f